In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [2]:
folder = 'C:/Users/jullian/Desktop/Jullian/Jullian/Data/DANS/OViN_2011-2017_Final/OViN_2011-2017_Final/'
filename = 'OVIN_2011-2017.csv'
path = folder + filename
ovin = pd.read_csv(path,na_values = ['#NULL!',' '])

C:\Users\jullian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (90,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#ovin['OPID'] = ovin['OPID'].astype('str')
#ovin['Jaar'] = ovin['Jaar'].astype('str')
#ovin['OPID2'] = ovin['OPID'].str.cat(ovin['Jaar'])
#print(len(ovin['OPID2']))
#print(len(ovin['OPID2'].unique()))
#print(len(ovin['OPID'].unique()))

conversion_list = ['OPID','VerplID','VerplNr','Doel','VertPC','AankPC','BuurtAdam','VertUur','VertMin','AankUur','AankMin','Jaar','Maand','Dag']
ovin[conversion_list] = ovin[conversion_list].fillna(np.NaN)
#ovin[conversion_list] = ovin[conversion_list].astype(float)
ovin[conversion_list] = ovin[conversion_list].astype('Int64')
ovin[conversion_list].dtypes

OPID         Int64
VerplID      Int64
VerplNr      Int64
Doel         Int64
VertPC       Int64
AankPC       Int64
BuurtAdam    Int64
VertUur      Int64
VertMin      Int64
AankUur      Int64
AankMin      Int64
Jaar         Int64
Maand        Int64
Dag          Int64
dtype: object

In [4]:
# This function creates a column which tells whether the individual made a trip in Amsterdam
def Amsterdam_trip(ovin):
    #ovin['VerplNr'] = pd.to_numeric(ovin['VerplNr'],errors = 'coerce')
    # add column with postal code area of a person's home
    #ovin['WoonPC'] = np.where(((ovin['Vertrekp']==int(1)) &  (ovin['VerplNr']==int(1))), ovin['VertPC'], np.NaN)
    ovin['Amsterdam_vpl'] = np.where(((ovin['VertGem']== 363) | (ovin['AankGem']==363)),1,np.NaN)
    # persons with same id have same home
    #print(ovin[['OPID','Amsterdam_vpl']])
    ovin['Amsterdam_vpl'] = ovin[['OPID','Amsterdam_vpl']].groupby('OPID').ffill()
    ovin['Amsterdam_vpl'] = ovin[['OPID','Amsterdam_vpl']].groupby('OPID').bfill()
    ovin['Amsterdam_vpl'] = ovin['Amsterdam_vpl'].fillna(0)
    #print(ovin[['OPID','Amsterdam_vpl']])
    return ovin

In [5]:
# This function creates 
def Car_trip(ovin):
    # add column with postal code area of a person's home
    #ovin['WoonPC'] = np.where(((ovin['Vertrekp']==int(1)) &  (ovin['VerplNr']==int(1))), ovin['VertPC'], np.NaN)
    ovin['car_trip'] = np.where((ovin['RVm']== 6),1,np.NaN)
    # persons with same id have same home
    ovin['car_trip'] = ovin[['OPID','car_trip']].groupby('OPID').ffill()
    ovin['car_trip'] = ovin[['OPID','car_trip']].groupby('OPID').bfill()
    ovin['car_trip'] = ovin['car_trip'].fillna(0)
    #print(ovin[['OPID','car_trip']])
    return ovin

In [6]:
def thuislocatie(ovin):
    ovin['home'] = np.where(((ovin['VerplNr'] == 1) & (ovin['Vertrekp'] == 1)),ovin['VertPC'],np.NaN)
    ovin['home'] = ovin[['OPID','home']].groupby('OPID').ffill()
    ovin['home'] = ovin['home'].fillna(0)
    return ovin

In [7]:
def Amsterdam_treinstation(ovin):
    ovin['Amsterdam_vertrekstation'] = np.where(((ovin['RVertStat'] >= 234) & (ovin['RVertStat'] <= 243))|(ovin['RVertStat'] == 250)|(ovin['RVertStat'] == 519)|(ovin['RVertStat'] == 602),ovin['RVertStat'],999)
    #ovin['Amsterdam_vertrekstation'] = ovin['Amsterdam_vertrekstation'].fillna(999)
    ovin['Amsterdam_aankomststation'] = np.where(((ovin['RAankStat'] >= 234) & (ovin['RAankStat'] <= 243))|(ovin['RAankStat'] == 250)|(ovin['RAankStat'] == 519)|(ovin['RAankStat'] == 602),ovin['RAankStat'],999)
    #ovin['Amsterdam_aankomststation'] = ovin['Amsterdam_aankomststation'].fillna(999)
    return ovin

In [8]:
temp = Amsterdam_trip(ovin)
temp = Car_trip(temp)
temp = thuislocatie(temp)
temp = Amsterdam_treinstation(temp)

temp['RVertStat'] = temp['RVertStat'].fillna(999)
temp['RVertStat'] = temp['RVertStat'].astype(int)
temp['RAankStat'] = temp['RAankStat'].fillna(999)
temp['RAankStat'] = temp['RAankStat'].astype(int)

In [9]:
l = temp[temp['Amsterdam_vpl']==1]
print(len(l['OPID'].unique())) # aantal individuen die een verplaatsing in Amsterdam heeft gemaakt.
l = l[l['car_trip']==1]
len(l['OPID'].unique()) #aantal individuen die met de auto een verplaatsing in Amsterdam hebben gedaan. 

15514


4966

In [18]:
Filtered_ovin = temp[((temp['Amsterdam_vpl']== 1) & (temp['car_trip']== 1))].copy()
PC4_Amsterdam = np.where(Filtered_ovin['BuurtAdam'] != 0,Filtered_ovin['home'],0)
PC4_Amsterdam = np.unique(PC4_Amsterdam)

# count how many individuals travelled to a specific AankPC 
count_AankPC = Filtered_ovin.groupby('AankPC').OPID.nunique()[PC4_Amsterdam]
count_VertPC = Filtered_ovin.groupby('VertPC').OPID.nunique()[PC4_Amsterdam]
count_BuurtAdam = Filtered_ovin.groupby('BuurtAdam').OPID.nunique()
count_Aankomststation = Filtered_ovin.groupby('Amsterdam_aankomststation').OPID.nunique()
count_Vertrekstation = Filtered_ovin.groupby('Amsterdam_vertrekstation').OPID.nunique()

# Filter the counts on amsterdam PC
#count_AankPC
#count_BuurtAdam
#count_AankPC
#count_Aankomststation
#count_Vertrekstation
#PC4_Amsterdam
#Filtered_ovin.groupby('AankPC').OPID.nunique()

#print(Filtered_ovin['AankPC'].value_counts())
#print(Filtered_ovin['BuurtAdam'].value_counts())

#count_AankPC.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Schedules_filtering\AankPC_count.csv')
#count_VertPC.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Schedules_filtering\VertPC_count.csv')
#count_BuurtAdam.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Schedules_filtering\BuurtAdam_count.csv')
#count_Aankomststation.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Schedules_filtering\Aankomststation_count.csv')
#count_Vertrekstation.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Schedules_filtering\Vertrekstation_count.csv')
#Filtered_ovin.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Schedules_filtering\Filtered_ovin.csv')

In [19]:
Filtered_ovin['AantRit'] = Filtered_ovin['AantRit'].astype(int)
Filtered_ovin['AantVpl'] = Filtered_ovin['AantVpl'].astype(int)

Filtered_ovin['vpl_weight'] = (1/Filtered_ovin['AantRit'])
Filtered_ovin['OPID_weight'] = (1/Filtered_ovin['AantVpl'])
Filtered_ovin

,OP,OPID,Steekproef,Mode,HHPers,HHSam,HHPlOP,HHLft1,HHLft2,HHLft3,...,FactorH,FactorP,FactorV,Amsterdam_vpl,car_trip,home,Amsterdam_vertrekstation,Amsterdam_aankomststation,vpl_weight,OPID_weight
637,1,11103083,1,1,3,3,3,0,0,1,...,36.194284,108.582853,39203.679210,1.0,1.0,1423,999.0,999.0,1.0,0.333333
638,0,11103083,1,1,3,3,3,0,0,1,...,36.194284,108.582853,39203.679210,1.0,1.0,1423,999.0,999.0,1.0,0.333333
639,0,11103083,1,1,3,3,3,0,0,1,...,36.194284,108.582853,39203.679210,1.0,1.0,1423,999.0,999.0,1.0,0.333333
676,1,11104002,1,3,3,3,3,1,0,0,...,39.848190,119.544569,42974.160210,1.0,1.0,1464,999.0,999.0,1.0,0.333333
677,0,11104002,1,3,3,3,3,1,0,0,...,39.848190,119.544569,42974.160210,1.0,1.0,1464,999.0,999.0,1.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908349,0,17651064,4,1,2,2,3,0,0,0,...,199.696721,399.393441,141589.892907,1.0,1.0,2022,999.0,999.0,1.0,0.200000
908372,1,17650088,4,1,2,2,3,0,0,0,...,202.801760,405.603520,144067.123346,1.0,1.0,2041,999.0,999.0,1.0,0.250000
908373,0,17650088,4,1,2,2,3,0,0,0,...,202.801760,405.603520,144067.123346,1.0,1.0,2041,999.0,999.0,1.0,0.250000
908374,0,17650088,4,1,2,2,3,0,0,0,...,202.801760,405.603520,144067.123346,1.0,1.0,2041,999.0,999.0,1.0,0.250000


In [12]:
def recode_doel(doel):
    doel = doel.astype(str)
    doel[doel=='1'] = 'naar huis'
    doel[doel=='2'] = 'Werken'
    doel[doel=='3'] = 'zakelijk bezoek in werksfeer'
    doel[doel=='4'] = 'vervoer als beroep'
    doel[doel=='5'] = 'afhalen/brengen personen'
    doel[doel=='6'] = 'afhalen/brengen goederen'
    doel[doel=='7'] = 'onderwijs/cursus volgen'
    doel[doel=='8'] = 'winkelen/boodschappen'
    doel[doel=='9'] = 'visite/logeren'
    doel[doel=='10'] = 'toeren/wandelen'
    doel[doel=='11'] = 'sport/hobby'
    doel[doel=='12'] = 'overige vrijetijdsbesteding'
    doel[doel=='13'] = 'diensten/persoonlijke verzorging'
    doel[doel=='14'] = 'ander doel'
    return doel
        

In [20]:
Filtered_ovin['doel_recoded'] = recode_doel(Filtered_ovin['Doel'].copy())
#pd.crosstab(Filtered_ovin['doel_recoded'],Filtered_ovin['AankPC'])[PC4_Amsterdam]
Filtered_ovin['doel_recoded']

637                       sport/hobby
638       overige vrijetijdsbesteding
639                         naar huis
676             winkelen/boodschappen
677                            Werken
                     ...             
908349                      naar huis
908372          winkelen/boodschappen
908373          winkelen/boodschappen
908374          winkelen/boodschappen
908375                      naar huis
Name: doel_recoded, Length: 20652, dtype: object

In [64]:
def compute_activity_time(df):
    df['Actduur2'] = df['Actduur'].copy()
    for i in range(0,(len(df)-1)):
        if df['VerplID'].iloc[i] == df['VerplID'].iloc[i-1]: # If its the same VerplID we have the same action duration as the previous
            df['Actduur2'].iloc[i] = df['Actduur2'].iloc[i-1]
        elif df['OPID'].iloc[i] != df['OPID'].iloc[i+1]: # can we compute the activity time? if its the last trip we cant
            df['Actduur2'].iloc[i] = pd.NaT
        elif pd.isnull(df['VertTijd'].iloc[i+1]): # if the next record does not have a arrival/departure time skip a record
            df['Actduur2'].iloc[i] = df['VertTijd'].iloc[i+2]-df['AankTijd'].iloc[i]
        elif pd.isnull(df['AankTijd'].iloc[i]): # If the record itself does not have an arrival/departure time take previous activity duration
            df['Actduur2'].iloc[i] = df['Actduur2'].iloc[i-1]
        else: 
            df['Actduur2'].iloc[i] = df['VertTijd'].iloc[i+1]-df['AankTijd'].iloc[i]
    df['Actduur2']= pd.to_timedelta(df['Actduur2'])
    df['Actduur2'] = pd.to_numeric((df['Actduur2'].dt.total_seconds()/60))
    df['Actduur2'] = np.where(df['Actduur2']<0,df['Actduur'],df['Actduur2'])
    return df

In [63]:
sample = Filtered_ovin[Filtered_ovin['OPID'] == 11201099]
sample['Actduur2']
sample[['AankTijd','VertTijd','VerplID','OPID','Actduur','Actduur2']]
#pd.isnull(sample['VertTijd'])
#sample['VertTijd'].shift(-1)

10987    False
10988    False
10989    False
10990     True
10991    False
10992    False
10993    False
Name: VertTijd, dtype: bool

In [65]:
Filtered_ovin['AankUur'] = np.where(Filtered_ovin['AankUur']>23,Filtered_ovin['AankUur']-24,Filtered_ovin['AankUur'])
Filtered_ovin['VertUur'] = np.where(Filtered_ovin['VertUur']>23,Filtered_ovin['VertUur']-24,Filtered_ovin['VertUur'])

Filtered_ovin['Datum'] = Filtered_ovin['Jaar'].astype(str) + '-' + Filtered_ovin['Maand'].astype(str) + '-' + Filtered_ovin['Dag'].astype(str)
Filtered_ovin['VertTijd'] = Filtered_ovin['VertUur'].astype(str) + ':0' + Filtered_ovin['VertMin'].astype(str) + ':00'
Filtered_ovin['AankTijd'] = Filtered_ovin['AankUur'].astype(str) + ':0' + Filtered_ovin['AankMin'].astype(str) + ':00'

Filtered_ovin['Datum'] = pd.to_datetime(Filtered_ovin['Datum'])
Filtered_ovin['AankTijd'] = pd.to_datetime(Filtered_ovin['AankTijd'],errors = 'coerce')
Filtered_ovin['VertTijd'] = pd.to_datetime(Filtered_ovin['VertTijd'],errors = 'coerce')

Filtered_ovin = compute_activity_time(Filtered_ovin)

C:\Users\jullian\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [76]:
# This is to see which activity duration are computed compared to what is given.

Filtered_ovin['compare_result'] = np.isclose(Filtered_ovin['Actduur2'],Filtered_ovin['Actduur'],equal_nan = True)
#Filtered_ovin[['Actduur2','Actduur','compare_result']]
Filtered_ovin[['OPID','Actduur2','Actduur']][Filtered_ovin['compare_result'] == False]
#Filtered_ovin['Actduur2']
#Filtered_ovin['OPID'][Filtered_ovin['Actduur2'] == 0]
#Filtered_ovin.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Schedules_filtering\Filtered_ovin_Actduur2.csv')

,OPID,Actduur2,Actduur
22560,11228049,300.0,0.0
22561,11228049,300.0,0.0
23176,11229112,1345.0,NaN
23177,11229112,1345.0,NaN
30816,11247065,630.0,90.0
...,...,...,...
887661,17644028,1370.0,NaN
887662,17644028,1370.0,NaN
887663,17644028,1370.0,NaN
892951,17143081,330.0,30.0


In [192]:
#Filtered_ovin[['Datum','VertTijd','AankTijd']]
#Filtered_ovin['AankTijd'] - Filtered_ovin['VertTijd']

#Filtered_ovin['RVertTijd'] = Filtered_ovin['RVertUur'].astype(str) + ':0' + Filtered_ovin['RVertMin'].astype(str) + ':00'
#Filtered_ovin['RAankTijd'] = Filtered_ovin['RAankUur'].astype(str) + ':0' + Filtered_ovin['RAankMin'].astype(str) + ':00'
#Filtered_ovin['RAankTijd'] = pd.to_datetime(Filtered_ovin['RAankTijd'],errors = 'coerce')
#Filtered_ovin['RVertTijd'] = pd.to_datetime(Filtered_ovin['RVertTijd'],errors = 'coerce')

# Check whether the manually computed Activity duration is the same
# i parked this because i dont think this will gain us a real lot. the current 'actduur looks solid'
Filtered_ovin['Actduur2'] = np.where((Filtered_ovin['OPID'] == Filtered_ovin['OPID'].shift(-1)) &
                                     (Filtered_ovin['VerplID'] != Filtered_ovin['VerplID'].shift(-1)),
                                     #(Filtered_ovin['AantRit'] == 1),
                                     Filtered_ovin['VertTijd'].shift(-1)-Filtered_ovin['AankTijd'],
                                     pd.to_timedelta(Filtered_ovin['Actduur'],unit = 'm'))
Filtered_ovin['Actduur2'] = pd.to_numeric((Filtered_ovin['Actduur2'].dt.total_seconds()/60))
Filtered_ovin['Actduur2'] = Filtered_ovin[['VerplID','Actduur2']].groupby('VerplID').ffill()

#Filtered_ovin['Actduur3'] = np.where((Filtered_ovin['AankTijd'].isnull()) &
#                              (Filtered_ovin['OPID'].shift(1) == Filtered_ovin['OPID']),
#                             #(Filtered_ovin['AantRit'] == 1),
#                              Filtered_ovin['VertTijd'].shift(-1)-Filtered_ovin['AankTijd'].shift(1),
#                              pd.to_timedelta(Filtered_ovin['Actduur2'],unit = 'm'))
#Filtered_ovin['Actduur3'] = pd.to_numeric((Filtered_ovin['Actduur3'].dt.total_seconds()/60))
#Filtered_ovin['Actduur3'] = Filtered_ovin[['VerplID','Actduur3']].groupby('VerplID').ffill()

Filtered_ovin['compare_result'] = np.isclose(Filtered_ovin['Actduur2'],Filtered_ovin['Actduur'],equal_nan = True)
Filtered_ovin[['Actduur2','Actduur','compare_result']]
Filtered_ovin[['OPID','Actduur2','Actduur']][Filtered_ovin['compare_result'] == False]
#sum(Filtered_ovin['compare_result'])
#[Filtered_ovin['AankTijd'],Filtered_ovin['VertTijd'].shift()]
#shift(-1) = next line
#shift(1) = previous line

C:\Users\jullian\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jullian\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jullian\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,OPID,Actduur2,Actduur
10990,11201099,NaN,0.0
22560,11228049,NaN,0.0
22561,11228049,NaN,0.0
30816,11247065,NaN,90.0
30817,11247065,NaN,0.0
...,...,...,...
868098,17640014,NaN,60.0
868468,17243012,NaN,342.0
868469,17243012,NaN,135.0
892951,17143081,NaN,30.0


In [188]:
sample = Filtered_ovin[Filtered_ovin['OPID'] == 11228049 ]
sample['Actduur3'] = np.where((sample['AankTijd'].isnull()) &
                              (sample['OPID'].shift(1) == sample['OPID']),
                             #(Filtered_ovin['AantRit'] == 1),
                              sample['VertTijd'].shift(-1)-sample['AankTijd'].shift(1),
                              pd.to_timedelta(sample['Actduur2'],unit = 'm'))
print(sample['Actduur3'])
sample['Actduur2']
sample['AankTijd']
#(sample['AankTijd'].isnull()) & (sample['OPID'].shift(1) == sample['OPID'])

22560        NaT
22561   05:00:00
22562   16:30:00
22563        NaT
Name: Actduur3, dtype: timedelta64[ns]


C:\Users\jullian\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


22560   2020-01-15 00:30:00
22561                   NaT
22562   2020-01-15 06:15:00
22563   2020-01-15 23:40:00
Name: AankTijd, dtype: datetime64[ns]